## 1. Install Dependencies

In [1]:
# pip install tensorflow opencv-python numpy matplotlib scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import tensorflow as tf
from matplotlib import pyplot as plt


2023-12-05 10:49:05.429396: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-05 10:49:05.451096: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-05 10:49:05.564163: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-05 10:49:05.564230: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-05 10:49:05.579331: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [3]:
print(tf.version)

<module 'tensorflow._api.v2.version' from '/home/vipul/.local/lib/python3.10/site-packages/tensorflow/_api/v2/version/__init__.py'>


In [4]:
#pip list

In [5]:
# For using GPUS
gpus = tf.config.experimental.list_physical_devices('GPU') # Listing GPUs
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu,True)

2023-12-05 10:49:07.094325: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-05 10:49:07.148408: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


## 2. Load the Data

In [6]:
import numpy as np
import cv2

In [9]:
os.getcwd()

'/home/vipul/Vipul/ML/Skin Cancer Diagnosis/Skin-Cancer-Classification-using-CNN-/Notebooks'

In [8]:
data = tf.keras.utils.image_dataset_from_directory('Dataset',shuffle=True)

NotFoundError: Could not find directory Dataset

In [ ]:
# tf.compat.v1.enable_eager_execution()
print(tf.executing_eagerly())

In [ ]:
data_iterator = data.as_numpy_iterator()

In [ ]:
# Get a batch from the Iterator
batch = data_iterator.next()

In [ ]:
#Images represnted as Np array
batch[0].shape

In [ ]:
#Class 1 : Malignant
#Class 0 : Benign
batch[1]

In [ ]:
fig, ax = plt.subplots(ncols=4,figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

## 3. Scaling the Data

In [ ]:
# Scaling the Data into 0 or 1
data = data.map(lambda x, y: (x/255, y))

In [ ]:
data.as_numpy_iterator().next()

In [ ]:
batch[0].max()

In [ ]:
fig, ax = plt.subplots(ncols=4,figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

## 4. Splitting the Data

In [ ]:
# No. of Batches
len(data)

In [ ]:
train_size = int(len(data)*.8)
val_size = int(len(data)*.1)
test_size = int(len(data)*.1)

In [ ]:
# Print the Train, Validate and Test Dataset
print(train_size)
print(val_size)
print(test_size)
# Train + Validate + Test = No. of batches 
# Split your Data Accordingly above

In [ ]:
# Data Distribution: 

# Shuffle the Data before adding it here
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.take(train_size+val_size).take(test_size)

In [ ]:
len(train)

## 5. Building the model

In [ ]:
train

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
model = Sequential()

In [ ]:
#model.add(Conv2D(#filters,#filter-size, stride, activation='relu'))
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

# Add Dense layer with L2 regularization
model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.5))

# Add Dense layer with L2 regularization and sigmoid activation for binary classification
model.add(Dense(1, activation='sigmoid', kernel_regularizer=regularizers.l2(0.01)))

In [ ]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
model.summary()

## 6. Training the Model

In [ ]:
logdir = "logs"

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [ ]:
hist = model.fit(train, epochs=2, validation_data=val, callbacks=[tensorboard_callback]) 

## 7. Plotting Performance

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label = 'val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label = 'val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# Predictions on the validation set
predictions = (model.predict(val) > 0.5).astype(np.int32)

# True labels
true_labels = val_labels.astype(np.int32)

# Compute confusion matrix
cm = confusion_matrix(true_labels, predictions)

In [ ]:
# Plot the confusion matrix
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()

## 8. Evaluating the Performance

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
test

In [ ]:
for batch in test.as_numpy_iterator():
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
print(f'Precision:{pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

In [ ]:
img = cv2.imread('Testing/benign2.jpeg')
plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
plt.show()

## 9. Testing the Model

In [ ]:
img = cv2.imread('Testing/benign.jpeg')
plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
resize.shape

In [ ]:
np.expand_dims(resize, 0).shape

In [ ]:
yhat = model.predict(np.expand_dims(resize/255 , 0))

In [ ]:
yhat

In [ ]:
if yhat > 0.5 :
    print(f'Malignant')
elif yhat < 0.5 :
    print(f'Benign')

## 10. Saving the Model

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save(os.path.join('models','test.keras'))

In [ ]:
new_model = load_model('models/skin_cnn_4_88%.keras')

In [ ]:
new_model.predict(np.expand_dims(resize/255, 0))